In [86]:
#convert the kegg id to a number

        

In [89]:
convert_tax('k117_1_1	esi:Exig_1092	53.4	103	41	1	1	103	127	222	8.3e-23	111.7')

'Exiguobacterium sibiricum 255-15'

In [127]:
import re
with open('real_tax_convert.txt','r') as tax_id:
    converter={}
    for line in tax_id:
        
        toks=line.strip().split('\t')
        test=toks[1]
        #print(test)
        try:
            others=re.search(r'([a-z]+), .*; (.*)',test)
            kegg_id=others.group(1)
            NCBI_id=others.group(2)
            converter[kegg_id]=NCBI_id
        except:
            print(test)
            break
        #kegg_id=others.group(0)
        


In [125]:
if 'Cryobacterium arcticum PAMC 27867' in blah:
    print('yes')
else:
    print('no')

no


In [171]:
class Node(object):

    def __init__(self, taxonomy, node_id, parent_id, tax_level):
        """Represents a node within a tree.
        Args:
            taxonomy (str): taxonomy name or ID
            node_id (str): taxonomy ID
            parent_id (str): taxonomy ID of parent
            tax_level (str): the taxonomic level for this node_id
        """
        # the current node's string ID
        self.taxonomy = taxonomy
        # the current node's digit ID
        self.node_id = node_id
        self.parent_id = parent_id
        self.tax_level = tax_level

class Tree(object):

    def __init__(self, tree_file):
        """Builds reference dictionary of Taxonomy Name, Taxonomy ID, and Parent Taxonomy ID."""
        self.tree = defaultdict(dict)
        
        with open(tree_file) as tree_fh:
            for line in tree_fh:
                toks = line.strip().split("\t")
                if not toks[0] == '1' and not toks[2] == '1':
                    assert not toks[0] == toks[2]
                if not len(toks) == 4:
                    logging.warning("Line [%s] does not have ID, NAME, PARENTID, TAX LEVEL" % line.strip())
                    continue
                self.add_node(toks[1], toks[0], toks[2], toks[3])
                
                

    def add_node(self, taxonomy, node_id, parent_id, tax_level):
        """Adds node to tree dictionary.
        Args:
            taxonomy (str): the taxonomy name
            node_id (str): the taxonomy id
            parent_id (str): the parent's taxonomy id
            tax_level (str): the taxonomic level for this node_id
        """
        
        # taxonomy id to node mapping; ensures unique nodes despite non-unique names
        self.tree[node_id] = Node(taxonomy, node_id, parent_id, tax_level)
        

    def lca(self, taxonomies, threshold=1.):
        """Returns the taxonomy of the LCA and optionally only use the top fraction of hits.
        Args:
            taxonomies (list): list of taxonomies (ID); when using threshold < 1 they
                should be ordered by decreasing bitscore
            threshold (Optional[float]): 0-1; threshold fraction of hits to be factored into lca
        Returns:
            str: node id of LCA
        """
        if threshold > 1:
            threshold = 1
        elif threshold < 0.01:
            # 10% as the minimum
            threshold = 0.1

        count_target = len(taxonomies) * threshold
        count_taxonomies = defaultdict(int)

        for taxonomy in taxonomies:
            #print('tax',taxonomy) 
        
            try:
                current_taxonomy = self.tree[taxonomy].node_id
                #print(current_taxonomy)
            except AttributeError:
                # dict when key not present
                # taxonomy represented in the reference database, but is not present in the tree
                continue
                
            while not current_taxonomy == "1":
                #print('curtx',current_taxonomy)
                count_taxonomies[current_taxonomy] += 1
                if count_taxonomies[current_taxonomy] >= count_target:
                    #print('target found')
                    return self.tree[current_taxonomy].node_id
                    
                # traverse up tree
                current_taxonomy = self.tree[current_taxonomy].parent_id
        return "1"
    
    def taxonomic_lineage(self, taxonomy):
        """For a given taxonomy ID, return its lineage as a list of IDs.
        Args:
            taxonomy (str): taxonomy name or taxonomy ID
        Returns:
            list of lineage
        Example:
            >>> tree = Tree("ref/ncbi_taxonomy_tree.txt")
            >>> tree.taxonomic_lineage("135621")
            ['1', '131567', '99999999', '2', '1224', '1236', '72274', '135621']
        """
        # a large portion of ORFs
        if taxonomy == "1":
            return [taxonomy]

        lineage = [taxonomy]
        while not taxonomy == "1":
            taxonomy = self.tree[taxonomy].parent_id
            # prepend
            lineage.insert(0, taxonomy)
        return lineage
    
    def lca_majority(self, taxonomy_list, majority_cutoff):
        """Finds a consensus majority up a tree structure.
        Args:
            taxonomy_list (list): list of taxonomy names or IDs
            majority_cutoff (float): this is the total length of the taxonomy list * majority fraction
        Returns:
            list: string representing majority taxonomy ID, lineage counts dictionary with a key
                of taxonomy ID and value of list populated with ordered lineage taxonomy IDs
        Example:
            >>> taxonomy_list = ['gamma subgroup', 'RNA similarity group I',
                                 'purple photosynthetic bacteria and relatives',
                                 'not Bacteria Haeckel 1894',
                                 'purple photosynthetic bacteria and relatives', 'gamma subgroup',
                                 'gamma subgroup', 'purple photosynthetic bacteria and relatives',
                                 'purple photosynthetic bacteria and relatives']
            >>> majority_cutoff = len(taxonomy_list) * .5
            >>> tree = Tree("ref/ncbi_taxonomy_tree.txt")
            >>> tree.lca_majority(taxonomy_list)
            ('1224', {'1224': ['1', '131567', '99999999', '2', '1224'],
                      '1236': ['1', '131567', '99999999', '2', '1224', '1236'],
                      '2': ['1', '131567', '99999999', '2'],
                      '286': ['1', '131567', '99999999', '2', '1224', '1236', '72274', '135621', '286']})
        """
        lineage_counts = Counter()
        lineages = {}
        for taxonomy in taxonomy_list:
            lineage = self.taxonomic_lineage(taxonomy)
            lineage_counts.update(lineage)
            lineages[taxonomy] = lineage
        lineage_indexes = index_of_list_items(lineages.values())
        for taxonomy in lineage_indexes:
            if lineage_counts[taxonomy] > majority_cutoff:
                return taxonomy, lineages
        return "1", lineages




In [174]:
test=Tree('refseq.tree')
lists=['6','10']
testing=test.lca(taxonomies)
print(testing)
test.taxonomic_lineage(testing)

1783272


['1', '131567', '2', '1783272']

In [166]:
from collections import Counter, defaultdict, deque, OrderedDict
import re


#this converts the kegg ID into the NCBI ID 
def convert_tax(aligned):
    
    with open('real_tax_convert.txt','r') as tax_id:
        converter={}
        for line in tax_id:

            toks=line.strip().split('\t')
            test=toks[1]
            #print(test)
            others=re.search(r'([a-z]+), .*; (.*)',test)
            kegg_id=others.group(1)
            NCBI_id=others.group(2)
            converter[kegg_id]=NCBI_id
        
        toks=aligned.strip().split('\t')
        kegg_id=toks[1].split(':')[0]
        try:

            if kegg_id in converter:
                return(converter[kegg_id])
        except KeyError:
            pass
                
            



#below will make the taxonomy name to the node id so that lca can run. somehow need to integrate into the Tree class(if possible)
tax_to_node_id={}
with open('refseq.tree') as tree_fh:
    for line in tree_fh:
        toks = line.strip().split("\t")
        if not toks[0] == '1' and not toks[2] == '1':
            assert not toks[0] == toks[2]
        if not len(toks) == 4:
            logging.warning("Line [%s] does not have ID, NAME, PARENTID, TAX LEVEL" % line.strip())
            continue
        tax_to_node_id[toks[1]]=toks[0]
        

#this function will generate the taxonomy that we want from the alignment file
def make_tax_list(alignment_file):

    with open(alignment_file,'r') as alignments:
        taxonomies=[]
        for line in alignments:
            converted_line=convert_tax(line)
            try:
                node_id=tax_to_node_id[converted_line]
            except KeyError:
                continue
            taxonomies.append(node_id)
        return taxonomies




    
        


In [193]:
from itertools import groupby
with open('head_20.txt','r') as alignments:
    for aligned, kegg_id in groupby(alignments, key=lambda x: x.split("\t")[0]):
        print(aligned)
        taxonomies=[]
        for item in kegg_id:
            converted_line=convert_tax(item)
            try:
                node_id=tax_to_node_id[converted_line]
            except KeyError:
                continue
            taxonomies.append(node_id)
        lca=test.lca(taxonomies)
        lineages=test.taxonomic_lineage(lca)
        print(lineages)
            
            

k117_1_1
['1', '131567', '2', '1783272']
k117_2_1
['1', '131567', '2', '1224', '28211', '356', '69277', '28100', '1867719']
k117_3_1
['1', '131567', '2', '1224', '28211', '356', '45401', '29407', '674703']


In [167]:
test.lca(taxonomies)

'1783272'

In [1]:
class DiamondHits(object):

    def __init__(self, names=None, max_hits=10, top_fraction=None):
        """Class that represents BLAST hits for a single target sequence. Hits are added to queues
        for bitscore and ID and ordered by increasing bitscore.
        Args:
            names (Optional[list]): when initiated with a name list; :func:`best_hit` and
                :func:`add` will no longer operate as intended
            max_hits (int): maximum number of hits to consider for this :class:`BlastHits` group
            top_fraction (float): fraction cutoff from best bitscore, e.g. 0.7 will filter out 699 when best bitscore is 1000
        Notes:
            max_hits and top_fraction work in conjunction of one another
        Examples:
            >>> hits = BlastHits()
            >>> hits.add("name_1", 100)
            >>> hits.add("name_2", 110)
            >>> hits.add("name_3", 120)
            >>> hits.add("name_4", 130)
            >>> hits.add("name_4", 140)
            >>> hits.majority()
            'name_4'
            >>> hits.best_hit()
            'name_4'
            >>> hits.add("name_5", 150)
            >>> hits.best_hit()
            'name_5'
            # demo max hits and top fraction
            >>> hits = BlastHits(max_hits=3, top_fraction=0.7)
            >>> hits.add("name_1", 100)
            # filtered hit
            >>> hits.add("name_2", 65)
            >>> hits.names
            deque(['name_1'])
            >>> hits.add("name_3", 70)
            deque(['name_3', 'name_1'])
            >>> hits.add("name_4", 80)
            >>> hits.add("name_5", 85)
            >>> hits.names
            deque(['name_4', 'name_5', 'name_1'])
            # new best bitscore
            >>> hits.add("name_6", 125)
            >>> dict(zip(hits.names, hits.bitscores))
            {'name_1': 100.0, 'name_6': 125.0}
        """
        if names is None:
            # increasing bitscore sorted
            self.names = deque()
            self.bitscores = deque()
        else:
            self.names = names
        self.max_hits = max_hits
        self.top_fraction = top_fraction

    def __repr__(self):
        return "{cls}[{tax}]".format(cls=self.__class__.__name__, tax=self.names)

    def __len__(self):
        return len(self.names)

    def add(self, name, bitscore):
        """Add entry to this :class:`BlastHits` group.
        Args:
            name (str): hit identifier
            bitscore (str): bitscore for hit
        """
        bitscore = float(bitscore)

        if self.top_fraction and self.bitscores:
            # the filter
            if bitscore < (self.bitscores[-1] * self.top_fraction):
                bitscore = None
            # new best
            elif bitscore > self.bitscores[-1]:
                score = self.bitscores[0]
                while score < bitscore * self.top_fraction:
                    self.names.popleft()
                    self.bitscores.popleft()
                    score = self.bitscores[0]

        if bitscore:
            # insert into sorted list
            idx = bisect.bisect_left(self.bitscores, bitscore)
            self.bitscores.insert(idx, bitscore)
            self.names.insert(idx, name)
            if len(self.names) > self.max_hits:
                # remove lowest bitscore
                self.names.popleft()
                self.bitscores.popleft()

    def best_hit(self):
        """Returns the hit ID of the best scoring alignment."""
        return self.names[-1]

    def majority(self):
        """Returns the hit ID of the best scoring hit ID that is repeated or the best hit when
        no items are repeated.
        """
        # no repeated names
        if len(self.names) == len(set(self.names)):
            return self.best_hit()
        else:
            # count each taxonomy, grab top taxonomy
            most_common = Counter(self.names).most_common(1)[0][0]
            # need to flip to grab best bitscore
            names_reversed = self.names.copy()
            names_reversed.reverse()
            # left most index match
            idx = names_reversed.index(most_common)
            return names_reversed[idx]

